# Metrics, validation strategies and baselines

В данном jupyter notebook рассматриваются примеры того, какие схемы валидации и метрики используются в рекомендательных системах.
Также построим простые модели (бейзлайны) на данных МТС Библиотеки. 

* [Preprocessing](#preprocessing)
* [General remarks](#general-remarks)
* [Metrics](#metrics)
    * [Regression](#regression)
    * [Classification](#classification)
    * [Ranking](#ranking)
* [Validation strategies](#validation)
* [Baselines](#baselines)

In [22]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise



<a id="preprocessing"></a>
# Preprocessing

Загрузим наши данные, теперь уже с фичами, и применим знания из [pandas-scipy-for-recsys](https://www.kaggle.com/sharthz23/pandas-scipy-for-recsys)

In [23]:
df = pd.read_pickle('./interactions_preprocessed.pickle')
df_users = pd.read_pickle('./users_preprocessed.pickle')
df_items = pd.read_pickle('./items_preprocessed.pickle')

In [24]:
df_users

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0
...,...,...,...
142883,159606,25_34,0.0
142884,159607,25_34,NaN
142885,159609,18_24,0.0
142886,159610,35_44,0.0


In [25]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


In [26]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня',
        'Боря', 'Боря', 'Боря',
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени',
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Искра жизни', 'Зеленая миля',
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Искра жизни,2
8,Вася,Зеленая миля,3


In [27]:
df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')


rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Зеленая миля                          3.0
        Рита Хейуорт и спасение из Шоушенка   NaN

In [28]:
df_merged = df_true.merge(df_recs, on = ['user_id','item_id'], how = 'left')

In [29]:
df_merged['hit@2'] = df_merged['rank'] <= 2

In [30]:
df_merged

,user_id,item_id,rank,hit@2
0,Аня,Мастер и Маргарита,NaN,False
1,Боря,451° по Фаренгейту,1.0,True
2,Вася,Зеленая миля,3.0,False
3,Вася,Рита Хейуорт и спасение из Шоушенка,NaN,False


In [31]:
df_merged['hit@2/2'] = df_merged['hit@2'] / 2
df_merged

,user_id,item_id,rank,hit@2,hit@2/2
0,Аня,Мастер и Маргарита,NaN,False,0.0
1,Боря,451° по Фаренгейту,1.0,True,0.5
2,Вася,Зеленая миля,3.0,False,0.0
3,Вася,Рита Хейуорт и спасение из Шоушенка,NaN,False,0.0


In [32]:
df_prec2 = df_merged.groupby('user_id').agg({'hit@2/2':'sum'}).reset_index()


In [33]:
df_prec2

,user_id,hit@2/2
0,Аня,0.0
1,Боря,0.5
2,Вася,0.0


In [34]:
print('Precision@2' ,df_prec2['hit@2/2'].mean())

Precision@2 0.16666666666666666


In [35]:
for k in [1, 2, 3]:
    hit_k = f'hit@{k}'
    hit_k2 = f'hit@{k}/{k}'
    df_merged[hit_k] = df_merged['rank'] <= k
    df_merged[hit_k2] = df_merged[hit_k] / k

In [64]:
users_count = df_merged['user_id'].nunique()

In [65]:
print( 'Precision@1', df_merged['hit@1/1'].sum() / users_count,'\n',
    'Precision@2', df_merged['hit@2/2'].sum() / users_count,'\n',
       'Precision@3', df_merged['hit@3/3'].sum() / users_count
       )

Precision@1 0.3333333333333333 
 Precision@2 0.16666666666666666 
 Precision@3 0.2222222222222222


In [47]:
df_merged

,user_id,item_id,rank,hit@2,hit@2/2,hit@1,hit@1/1,hit@3,hit@3/3
0,Аня,Мастер и Маргарита,NaN,False,0.0,False,0.0,False,0.000000
1,Боря,451° по Фаренгейту,1.0,True,0.5,True,1.0,True,0.333333
2,Вася,Зеленая миля,3.0,False,0.0,False,0.0,True,0.333333
3,Вася,Рита Хейуорт и спасение из Шоушенка,NaN,False,0.0,False,0.0,False,0.000000


In [59]:
df_merged['hit@2'].sum()/df_merged['hit@2'].count()

0.25

In [66]:
df_merged = df_merged.merge(df_merged.groupby(['user_id']).agg({'item_id':'count'}).reset_index().rename(columns = {'item_id':'users_item_count'}) , on = 'user_id', how = 'left')

In [67]:
df_merged

,user_id,item_id,rank,hit@2,hit@2/2,hit@1,hit@1/1,hit@3,hit@3/3,users_item_count
0,Аня,Мастер и Маргарита,NaN,False,0.0,False,0.0,False,0.000000,1
1,Боря,451° по Фаренгейту,1.0,True,0.5,True,1.0,True,0.333333,1
2,Вася,Зеленая миля,3.0,False,0.0,False,0.0,True,0.333333,2
3,Вася,Рита Хейуорт и спасение из Шоушенка,NaN,False,0.0,False,0.0,False,0.000000,2


In [71]:
print( (df_merged['hit@1'] / df_merged['users_item_count']).sum() / users_count)

0.3333333333333333


In [72]:
print( (df_merged['hit@2'] / df_merged['users_item_count']).sum() / users_count)

0.3333333333333333


In [74]:
print( (df_merged['hit@3'] / df_merged['users_item_count']).sum() / users_count)

0.5


Mean Average Precision
То есть MAP - это усреднение AveragePrecision по всем пользователям. А AveragePrecision в свою очередь, это средний Precision@K по релевантным объектам одного пользователя

In [75]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


In [76]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня',
        'Боря', 'Боря', 'Боря',
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени',
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Рита Хейуорт и спасение из Шоушенка', 'Зеленая миля',
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Рита Хейуорт и спасение из Шоушенка,2
8,Вася,Зеленая миля,3


In [79]:
df_merged = df_true.merge(df_recs,on = ['user_id', 'item_id'], how='left')


,user_id,item_id,rank
0,Аня,Мастер и Маргарита,NaN
1,Боря,451° по Фаренгейту,1.0
2,Вася,Зеленая миля,3.0
3,Вася,Рита Хейуорт и спасение из Шоушенка,2.0


In [80]:
df_merged.sort_values(by=['user_id', 'rank'] , inplace= True)

In [82]:
df_merged['reciprocal_rank'] = 1 / df_merged['rank']

In [86]:
df_merged = df_merged.reset_index()

In [90]:
df_merged[['reciprocal_rank']] = df_merged[['reciprocal_rank']].fillna(0)

In [91]:
df_merged

,user_id,item_id,rank,reciprocal_rank
0,Аня,Мастер и Маргарита,NaN,0.000000
1,Боря,451° по Фаренгейту,1.0,1.000000
2,Вася,Рита Хейуорт и спасение из Шоушенка,2.0,0.500000
3,Вася,Зеленая миля,3.0,0.333333


In [92]:
mrr = df_merged.groupby(['user_id']).agg({'reciprocal_rank':'max'}).reset_index()

In [93]:
mrr

,user_id,reciprocal_rank
0,Аня,0.0
1,Боря,1.0
2,Вася,0.5
